---

### Memory

---

#### Conversation Buffer Memory
- 지금까지의 대화 내용 전체를 저장하는 메모리
- 대화 내용이 길어질 수록 메모리가 계속해서 커지기 때문에 비효율적
    - 이전의 대화 내용 전체를 프롬프트로 AI에게 보내기 때문

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    {"input":"Hi"}
    , {"output":"How are you?"}
)

memory.load_memory_variables({})

# Save_context를 반복 실행할 수록 메모리에 저장되는 데이터가 늘어남
# Load_memory_variables를 통해 확인할 수 있음

{'history': [HumanMessage(content='Hi'), AIMessage(content='How are you?')]}

---

#### Conversation Buffer Window Memory
- Conversation Buffer Memory 중 (Window에 들어온) 일부만을 저장하는 메모리
    - 예를 들어, 최근 5 개의 메시지를 저장하는 방식이면 6 번째 메시지가 들어오면 최초의 메시지는 지워짐
- 모든 대화를 저장하지 않아도 되는 장점이 있음
- 챗봇이 최근 대화에만 집중하는 단점이 있음

In [15]:
from langchain.memory import ConversationBufferWindowMemory

memory_window = ConversationBufferWindowMemory(return_messages=True
                                               , k=5 # 최근 5개의 데이터만 유지, 또는 window_size=5 로도 가능
                                               )

def add_message(input, output):
    memory_window.save_context({"input":input}, {"output":output})

def get_history():
    return memory_window.load_memory_variables({})

In [16]:
for i in range(10):
    add_message(i,i)
    print(get_history())

# 최근 5개의 데이터만 유지되는 것을 확인할 수 있음

{'history': [HumanMessage(content='0'), AIMessage(content='0')]}
{'history': [HumanMessage(content='0'), AIMessage(content='0'), HumanMessage(content='1'), AIMessage(content='1')]}
{'history': [HumanMessage(content='0'), AIMessage(content='0'), HumanMessage(content='1'), AIMessage(content='1'), HumanMessage(content='2'), AIMessage(content='2')]}
{'history': [HumanMessage(content='0'), AIMessage(content='0'), HumanMessage(content='1'), AIMessage(content='1'), HumanMessage(content='2'), AIMessage(content='2'), HumanMessage(content='3'), AIMessage(content='3')]}
{'history': [HumanMessage(content='0'), AIMessage(content='0'), HumanMessage(content='1'), AIMessage(content='1'), HumanMessage(content='2'), AIMessage(content='2'), HumanMessage(content='3'), AIMessage(content='3'), HumanMessage(content='4'), AIMessage(content='4')]}
{'history': [HumanMessage(content='1'), AIMessage(content='1'), HumanMessage(content='2'), AIMessage(content='2'), HumanMessage(content='3'), AIMessage(content='3'),

---

#### Conversation Summary Memory
- llm을 사용함

In [22]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory_summary = ConversationSummaryMemory(llm=llm, return_messages=True) 
# llm을 통해 memory를 사용하므로 비용이 발생함
# ConversationSummrayMemory는 대화 내용을 요약하여 저장함
# 짧은 대화에서는 초기 비용이 발생하지만, 대화가 길어질수록 비용이 줄어듦

def add_message_(input, output):
    memory_summary.save_context({"input":input}, {"output":output})

def get_history_():
    return memory_summary.load_memory_variables({})

add_message_("Hi I'm Minseop Ji, I live in South Korea", "Hi Minseop, that is so cool!")

In [23]:
add_message_("South Korea is so pretty", "I wish I could go!!!")

In [24]:
get_history_()

{'history': [SystemMessage(content='The human introduces themselves as Minseop Ji from South Korea. The AI responds by saying that it thinks that is cool and expresses a desire to visit South Korea because it is so pretty.')]}

---

#### Conversation Summary Buffer Memory
- Conversation Summary Memory와 Conversation Buffer Memory의 결합 모델
- 메모리에 메시지의 개수를 저장하고, 저장 한도를 넘어설 때 기존의 메모리를 삭제하는 것 대신 Summary를 통해 요약 저장하여 보관

In [30]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory_summary_buffer = ConversationSummaryBufferMemory(llm=llm, return_messages=True, max_token_limit=50)
# token limit을 50으로 설정하면, 50개의 토큰만 유지하고 나머지는 삭제함

def add_message__(input, output):
    memory_summary_buffer.save_context({"input":input}, {"output":output})

def get_history__():
    return memory_summary_buffer.load_memory_variables({})

In [31]:
add_message__("Hi I'm Minseop Ji, I live in South Korea", "Hi Minseop, that is so cool!")
get_history__()

{'history': [HumanMessage(content="Hi I'm Minseop Ji, I live in South Korea"),
  AIMessage(content='Hi Minseop, that is so cool!')]}

In [32]:
add_message__("South Korea is so pretty", "I wish I could go!!!")
get_history__()

{'history': [SystemMessage(content='The human introduces themselves as Minseop Ji from South Korea.'),
  AIMessage(content='Hi Minseop, that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [33]:
add_message__("How far is Korea from Argentina?", "I'm not sure, but I think it's pretty far")
get_history__()

{'history': [SystemMessage(content="The human introduces themselves as Minseop Ji from South Korea. The AI responds by saying it's cool and the human mentions that South Korea is pretty."),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I'm not sure, but I think it's pretty far")]}

* SystemMessage로 요약되어 memory에 저장됨

---
#### Conversation Knowledge Graph Memory

- 마찬가지로 llm을 사용하는 Memory
- 대화 중에 Entity의 Knowledge Graph를 생성함

In [34]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory_kg = ConversationKGMemory(llm=llm, return_messages=True)

def add_message___(input, output):
    memory_kg.save_context({"input":input}, {"output":output})

def get_history___():
    return memory_kg.load_memory_variables({})

In [37]:
add_message___("Hi I'm Minseop Ji, I live in South Korea", "Hi Minseop, that is so cool!")
memory_kg.load_memory_variables({"input":"Who is Minseop Ji?"})

{'history': [SystemMessage(content='On Minseop Ji: Minseop Ji lives in South Korea. Minseop Ji is a person.')]}

- 특정 Entity (Minseop Ji)에 대한 성격을 정의하고 답변을 작성함

In [38]:
add_message___("Minseop Ji likes kimchi.", "Minseop, that is so cool!")
memory_kg.load_memory_variables({"input":"what does Minseop Ji like?"})

{'history': [SystemMessage(content='On Minseop Ji: Minseop Ji lives in South Korea. Minseop Ji is a person. Minseop Ji likes kimchi.')]}

---

### Chain To use Memory

---
#### llm chain
- off-the-shelf(일반 목적 chain)의 일종
- 이미 만들어져 있는 chain인데 직접 모델을 만들 경우 부적합한 경우가 많음
- 

In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)
memory_summary_buffer = ConversationSummaryBufferMemory(
    llm=llm
    , return_messages=True
    , max_token_limit=80
    , memory_key="chat_history" # memory_key를 지정하면, 해당 key에 대한 prompt를 구성함
)

# 챗봇이 이전의 대화 내역을 기억하도록 prompt를 구성함
template = """
You are a helpful AI talking to a human.

{chat_history}
Human: {question}
You:
"""

# memory_summary_buffer를 LLMChain에 적용
chain = LLMChain(
    llm=llm
    , memory=memory_summary_buffer
    # , prompt = PromptTemplate.from_template("{question}") 이렇게 구현할 경우 prompt에 history가 포함되지 않아 챗봇이 이전의 대화 내역을 기억하지 못함
    , prompt = PromptTemplate.from_template(template)
    , verbose=True # chain의 prompt log를 출력함
)

chain.predict(question = "My name is Minseop Ji.")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human.

[]
Human: My name is Minseop Ji.
You:


> Finished chain.


'Hello Minseop Ji! How can I assist you today?'

In [6]:
chain.predict(question = "I live in Cheongju, South Korea.")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human.

[HumanMessage(content='My name is Minseop Ji.'), AIMessage(content='Hello Minseop Ji! How can I assist you today?')]
Human: I live in Cheongju, South Korea.
You:


> Finished chain.


"That's great! Cheongju is a beautiful city in South Korea. How can I assist you today, Minseop Ji?"

In [7]:
chain.predict(question = "What is my name?")
# 이전에 입력한 대화 내용을 prompt에 반영하지 못해서 정확한 답변을 얻지 못함
# 반면 load_memory_variables에는 정상적으로 입력한 대화 내용이 반영 및 요약되어 있음



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human.

[SystemMessage(content='The human introduces themselves as Minseop Ji.'), AIMessage(content='Hello Minseop Ji! How can I assist you today?'), HumanMessage(content='I live in Cheongju, South Korea.'), AIMessage(content="That's great! Cheongju is a beautiful city in South Korea. How can I assist you today, Minseop Ji?")]
Human: What is my name?
You:


> Finished chain.


'Your name is Minseop Ji.'

---
#### Chat 형태로 변환

In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory_summary_buffer = ConversationSummaryBufferMemory(
    llm=llm
    , return_messages=True
    , max_token_limit=80
    , memory_key="chat_history" # memory_key를 지정하면, 해당 key에 대한 prompt를 구성함
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human.")
    , MessagesPlaceholder(variable_name="chat_history") # chat_history를 placeholder로 지정함
    , ("human", "{question}")
])

# memory_summary_buffer를 LLMChain에 적용
chain = LLMChain(
    llm=llm
    , memory=memory_summary_buffer
    , prompt = prompt
    , verbose=True # chain의 prompt log를 출력함
)

chain.predict(question = "My name is Minseop Ji.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Minseop Ji.

> Finished chain.


'Hello Minseop Ji! How can I assist you today?'

In [4]:
chain.predict(question = "I live in Cheongju, South Korea.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Minseop Ji.
AI: Hello Minseop Ji! How can I assist you today?
Human: I live in Cheongju, South Korea.

> Finished chain.


"That's great to know, Minseop Ji! Cheongju is a beautiful city in South Korea. Is there anything specific you would like to know or discuss about Cheongju?"

In [5]:
chain.predict(question = "What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
System: The human introduces themselves as Minseop Ji.
AI: Hello Minseop Ji! How can I assist you today?
Human: I live in Cheongju, South Korea.
AI: That's great to know, Minseop Ji! Cheongju is a beautiful city in South Korea. Is there anything specific you would like to know or discuss about Cheongju?
Human: What is my name?

> Finished chain.


'Your name is Minseop Ji.'

---
#### Lang Chain Expression 사용

In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory_summary_buffer = ConversationSummaryBufferMemory(
    llm=llm
    , return_messages=True
    , max_token_limit=80
    , memory_key="chat_history" # memory_key를 지정하면, 해당 key에 대한 prompt를 구성함
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human.")
    , MessagesPlaceholder(variable_name="chat_history") # chat_history를 placeholder로 지정함
    , ("human", "{question}")
])

def load_memory(input):
    print(input)
    return memory_summary_buffer.load_memory_variables({})["chat_history"]

# chain = prompt | llm 
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm 
# 이렇게 구현하면 load_memory를 통해 chat_history를 먼저 가져와서 chat_history에 할당

chain.invoke({
    # "chat_history": memory.load_memory_variables({})["chat_history"] <- 이렇게 구현할 경우 매 chain 호출마다 chat_history를 추가해야 함
    "question": "My name is Minseop Ji."
})
# load_memory에 input이 들어가고, load_memory의 출력이 chat_history에 할당됨
# 이후 prompt에 chat_history가 입력되고, prompt의 출력이 llm에 입력됨

{'question': 'My name is Minseop Ji.'}


AIMessage(content='Hello Minseop Ji! How can I assist you today?')